In [ ]:
#Dominik Kowalczyk 
#Lista 1

#Zrobić i uzupełnić na podstawie pseudokodu 

#Zadanie 1 - Regresja liniowa
import numpy as np
import pytest
from sklearn.linear_model import LinearRegression

class LinearRegr:
    def fit(self, X, Y):
        #X = np.array, shape = (n, m)
        n, m = X.shape

        #Wektor parametrów Beta musi mieć m+1 (mamy m dla liczby X oraz +1 dla wyrazu wolnego beta0)
        self.beta = np.zeros((m+1))

        #Uwaga: przed zastosowaniem wzoru do X nalezy dopisac kolumne zlozona z jedynek!
        X_jedynki = np.hstack((np.ones((n, 1)), X))

        #Znajduje beta minimalizujace kwadratowa funkcje kosztu L uzywajac wzoru.
        #Wzór z wykładu: beta = (X^T X)^(-1) X^T Y 
        A = X_jedynki.T @ X_jedynki
        b = X_jedynki.T @ Y

        #Rozwiązujemy i nadpisujemy wyniki (tam gdzie mamy m+1 zer) -->  A*beta = b
        self.beta = np.linalg.solve(A, b)
        return self
    

    def predict(self, X):
        #wejscie X = np.array, shape = (k, m)
        k, m = X.shape
        
        #Dodaj kolumnę jedynek do X
        X_jedynki = np.hstack((np.ones((k, 1)), X))
        
        #Wzór na prognoze - macierzowo X (z jedynkami) * beta
        Y_prognoza = X_jedynki @ self.beta

        #zwraca Y = wektor(f(X_1), ..., f(X_k)
        return Y_prognoza


def test_RegressionInOneDim():
    X = np.array([1,3,2,5]).reshape((4,1))
    Y = np.array([2,5, 3, 8])
    a = np.array([1,2,10]).reshape((3,1))
    expected = LinearRegression().fit(X, Y).predict(a)
    actual = LinearRegr().fit(X, Y).predict(a)
    assert list(actual) == pytest.approx(list(expected))

def test_RegressionInThreeDim():
    X = np.array([1,2,3,5,4,5,4,3,3,3,2,5]).reshape((4,3))
    Y = np.array([2,5, 3, 8])
    a = np.array([1,0,0, 0,1,0, 0,0,1, 2,5,7, -2,0,3]).reshape((5,3))
    expected = LinearRegression().fit(X, Y).predict(a)
    actual = LinearRegr().fit(X, Y).predict(a)
    assert list(actual) == pytest.approx(list(expected))

test_RegressionInOneDim()               #jak nic nie zwracają to wszystko jest tak jak powinno byc - assert 
test_RegressionInThreeDim()

In [4]:
#Zadanie 2 - iteracja Ridge
import numpy as np
import pytest
from sklearn.linear_model import Ridge

class RidgeRegr:
    def __init__(self, alpha, zmienna_uczaca=0.01, maks_iteracja=10000, tolerancja=1e-8):
        self.alpha = alpha  
        self.zmienna_uczaca = zmienna_uczaca        
        self.maks_iteracja = maks_iteracja
        self.tolerancja = tolerancja

        #B = (X^T * X + alfaI)^-1 * X^T *Y
    def fit(self, X, Y):
        n, m = X.shape
        
        #Dodaj kolumnę jedynek dla beta_0
        X_jedynki1 = np.hstack([np.ones((n, 1)), X])
        #tak samo jak w zadaniu 1 
        self.beta = np.zeros(m + 1)

        #funkcja kosztu L(θ) = (1/2)*Σ_{i=1}^{n} (y_i - f(x_i))^2 + α/2*Σ_{j=1}^{m} Beta^2
        #jak policzymy pochodna dostaniemy 1/2 * 2X^T(X*beta - Y)
        #gradient mozna podzielic na czesc regularyzacji (alfa/2 suma B^2)
        for i in range(self.maks_iteracja):
            y_prognoza = X_jedynki1 @ self.beta
            roznica = y_prognoza - Y                #minus w gadiencie do środka

            #w zeszycie napisane wyprowadzenie 
            #gradient błędu + regularyzacja (wspolczynniki oprocz beta0 bo to stala - czyli pomijamy 0 - stad [1:])
            gradient = (X_jedynki1.T @ roznica) + self.alpha * np.r_[0, self.beta[1:]]

            #zapamietujemy stare wagi (aby potencjalnie wrocic)
            stara_beta = self.beta.copy()
            #bierzemy nowa bete uaktualniona o nasz gradient i zmienna uczaca
            self.nowa_beta = self.beta - self.zmienna_uczaca * gradient
            #aktualizujemy na nowa
            self.beta = self.nowa_beta

            #nalezy kiedys przerwac, czyli wtedy gdy nasza minimalizacja funkcji kosztu jest bardzo mala, czyli praktycznie przestala sie uczyc
            #sprawdzamy klasyczna norma euklidesowa
            if np.linalg.norm(self.beta - stara_beta) < self.tolerancja:
                break   

        return self
    
    #funcja predict analogiczna do tej z zadania 1
    def predict(self, X):
        k, m = X.shape
        #dodajemy 1 - czyli aby byl wyraz wolny beta0 (kolumna 1 na poczatku)
        X_jedynki2 = np.hstack([np.ones((X.shape[0], 1)), X])
        Y_predykcja = X_jedynki2 @ self.beta
        return Y_predykcja

def test_RidgeRegressionInOneDim():
    X = np.array([1,3,2,5]).reshape((4,1))
    Y = np.array([2,5, 3, 8])
    X_test = np.array([1,2,10]).reshape((3,1))
    alpha = 0.3
    expected = Ridge(alpha).fit(X, Y).predict(X_test)
    actual = RidgeRegr(alpha).fit(X, Y).predict(X_test)
    assert list(actual) == pytest.approx(list(expected), rel=1e-5)

def test_RidgeRegressionInThreeDim():
    X = np.array([1,2,3,5,4,5,4,3,3,3,2,5]).reshape((4,3))
    Y = np.array([2,5, 3, 8])
    X_test = np.array([1,0,0, 0,1,0, 0,0,1, 2,5,7, -2,0,3]).reshape((5,3))
    alpha = 0.4
    expected = Ridge(alpha).fit(X, Y).predict(X_test)
    actual = RidgeRegr(alpha).fit(X, Y).predict(X_test)
    #assert list(actual) == pytest.approx(list(expected), rel=1e-3)
    print(expected)
    print(actual)
test_RidgeRegressionInOneDim()                 
test_RidgeRegressionInThreeDim()


[ 0.54685378 -1.76188321  1.58691716  5.15527388  3.66704391]
[ 0.54684663 -1.76188798  1.58691183  5.15528137  3.66704223]
